Find and draw largest contour:  
https://stackoverflow.com/questions/44588279/find-and-draw-the-largest-contour-in-opencv-on-a-specific-color-python  

Straighten a crooked image:  
https://stackoverflow.com/questions/41995916/opencv-straighten-an-image-with-python  

Find all bounding boxes:  
https://stackoverflow.com/questions/21104664/extract-all-bounding-boxes-using-opencv-python  

Deskew images (This one has been the most helpful, the others...no):  
https://www.pyimagesearch.com/2017/02/20/text-skew-correction-opencv-python/  


In [1]:
from pathlib import Path
from itertools import tee
from os import path
import cv2
import numpy as np

IMG_SRC_DIR = "./jpg"

In [2]:
def load(img):
    # load the image from disk
    image = cv2.imread(str(img))

    # convert to grayscale, flip foreground and background
    # foreground is now "white" and the background is "black"
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)

    # threshold the image, setting all foreground pixels to 255 and all background pixels to 0
    return image, cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [3]:
def rotate(image, angle):
    # rotate the image to deskew it
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)


In [4]:
def save(img, img_path, save_dir):
    old_name = str(img_path.name)
    name = path.join(save_dir, old_name)
    cv2.imwrite(name, img)

In [5]:
# collect image paths
gen, image_paths = tee(Path(IMG_SRC_DIR).iterdir())
for img_path in image_paths:
    if img_path.suffix == ".jpg":
        image, thresh = load(img_path)
        
        # grab the (x, y) coordinates of all pixel values that
        # are greater than zero, then use these coordinates to
        # compute a rotated bounding box that contains all
        # coordinates
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]

        # the `cv2.minAreaRect` function returns values in the
        # range [-90, 0); as the rectangle rotates clockwise the
        # returned angle trends to 0 -- in this special case we
        # need to add 90 degrees to the angle
        if angle < -45:
            angle = -(90 + angle)
            
        # otherwise, just take the inverse of the angle to make it positive
        else:
            angle = -angle
        
        rotated_img = rotate(image, angle)
        save(rotated_img, img_path, IMG_SRC_DIR)
        
        # exaggerate the angle to set a cutoff point
        score = (angle*100)*(angle*100)
        if score > 1000:
            print(f"{str(img_path)}, angle: {angle}, score: {score}")
        else:
            print("Rotated: ", str(img_path))
        
        
        
        # draw the correction angle on the image so we can validate it
        # cv2.putText(rotated, "Angle: {:.2f} degrees".format(angle), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        # show the output image
        # print("[INFO] angle: {:.3f}".format(angle))
        # cv2.imshow("Input", image)
        # cv2.imshow("Rotated", rotated)
        # cv2.waitKey(0)
        
print("Finished rotating images.")

Rotated:  jpg/alphabet00794.jpg
Rotated:  jpg/alphabet00793.jpg
Rotated:  jpg/alphabet00787.jpg
Rotated:  jpg/alphabet00368.jpg
jpg/alphabet00369.jpg, angle: 0.6902772188186646, score: 4764.826388200305
Rotated:  jpg/alphabet00786.jpg
Rotated:  jpg/alphabet00792.jpg
Rotated:  jpg/alphabet00009.jpg
Rotated:  jpg/alphabet00790.jpg
Rotated:  jpg/alphabet00791.jpg
Rotated:  jpg/alphabet00785.jpg
Rotated:  jpg/alphabet00008.jpg
Rotated:  jpg/alphabet00005.jpg
Rotated:  jpg/alphabet00788.jpg
Rotated:  jpg/alphabet00367.jpg
Rotated:  jpg/alphabet00366.jpg
Rotated:  jpg/alphabet00789.jpg
Rotated:  jpg/alphabet00010.jpg
Rotated:  jpg/alphabet00004.jpg
Rotated:  jpg/alphabet00006.jpg
Rotated:  jpg/alphabet00364.jpg
Rotated:  jpg/alphabet00365.jpg
Rotated:  jpg/alphabet00007.jpg
Rotated:  jpg/alphabet00003.jpg
Rotated:  jpg/alphabet00361.jpg
Rotated:  jpg/alphabet00360.jpg
Rotated:  jpg/alphabet00002.jpg
Rotated:  jpg/alphabet00362.jpg
Rotated:  jpg/alphabet00363.jpg
Rotated:  jpg/alphabet00001.j